# Result Data Cleaning
### In this notebook we will be cleaning the data we extracted from the pdf in the data extraction notebook https://www.kaggle.com/code/mizerable/extract-pdf-data

Start by importig required libraries

In [ ]:
import pandas as pd
import numpy as np
import re

Importig the csv file in DataFrame

In [ ]:
df = pd.read_csv('./data/results_v2.csv',index_col=0)

In [ ]:
df.info()

#### clean status column
The status column values have a pipe | in front of the status we will remove this

In [ ]:
df['status'] = df['status'].map(lambda x: x[2:])

#### clean sgpi column and make it of correct type
We remove the extra characters from the sgpi column values and put 0.0 where the students are unsuccessful and have '--' as sgpi

In [ ]:
df['sgpi'] = df['sgpi'].map(lambda x: x.strip()[:4]) 
df['sgpi'] = df['sgpi'].map(lambda x: '0.0' if x == '-- -' else x)
df['sgpi'] = pd.to_numeric(df['sgpi'])

clean agg_marks to remove credits. everyone has 22. make it of correct type

In [ ]:
df['agg_marks'] = df['agg_marks'].map(lambda x: x.strip()[6:]) 
df['agg_marks'] = df['agg_marks'].map(lambda x: '0.0' if x == '--' else x)
df['agg_marks'] = pd.to_numeric(df['agg_marks'])

Making a new year of adimission column deducing the value based on the PRN as the first four digits of the prn are the year of admission

In [ ]:
df['year_of_admission'] = df['prn'].map(lambda x: int(str(x)[:4]))

#### clean centre column
the center column has the college id and the location of college both combined we will seperate them into their own columns

In [ ]:
df['clg_id'] = df['centre'].map(lambda x: x.split('-')[1])
df['clg_id'] = df['clg_id'].map(lambda x: x.strip('(').strip(')'))

In [ ]:
df['centre'] = df['centre'].map(lambda x: x.split('-')[0])
df['centre'] = df['centre'].map(lambda x: x.split(')')[1])

We will create a new gender column based on the data from name column as name beginning with '/' indicate female student.

In [ ]:
df['gender'] = df['name'].map(lambda x: 'F' if x[0]=='/' else 'M')

#### Create a seperate table for subject and marks 

In [ ]:
subject = df[['seat_no','sub1_10']].copy()

In [ ]:
subject['sub1_10'] = subject['sub1_10'].map(lambda x: x.strip('[').strip(']'))

seperating each subject into their own column

In [ ]:
cols = ['paper_1','paper_2','paper_3','paper_4','paper_5','paper_6','paper_7','paper_8','paper_9','paper_10']

for i in range(10):
    subject.loc[:,cols[i]] = subject['sub1_10'].map(lambda x: x.split(',')[i])

In [ ]:
subject.drop(['sub1_10'], axis=1, inplace =True)

Every subject code now has a ' around it so we will remove them for consistency.

In [ ]:
subject[cols] = subject[cols].apply(lambda x: x.map(lambda y: y.strip().strip("'")), axis =1) #TODO: ask stackoverflow if there is a better way to do this.

In [ ]:
subject.head()

In [ ]:
marks = df[['seat_no','marks1_10']].copy()

In [ ]:
marks['marks1_10'] = marks['marks1_10'].map(lambda x: x.strip('[').strip(']'))

seperating the marks of every paper into their own column

In [ ]:
cols = ['paper_1','paper_2','paper_3','paper_4','paper_5','paper_6','paper_7','paper_8','paper_9','paper_10']

for i in range(10):
    marks.loc[:,cols[i]] = marks['marks1_10'].map(lambda x: x.split(',')[i])

In [ ]:
marks.drop(['marks1_10'], axis=1, inplace =True)

We will go a step further and divide the mark of each paper into the external marks, internal marks and the total marks received.

In [ ]:
IA_cols = ['paper_1_IA','paper_2_IA','paper_3_IA','paper_4_IA','paper_5_IA','paper_6_IA','paper_7_IA','paper_8_IA','paper_9_IA','paper_10_IA']

marks[IA_cols] = marks[cols].apply(lambda x: x.map(lambda y: y[7:-11]) , axis =1) #UNKNOWN WHY ITS -11 but it works!!!!

In [ ]:
TOT_cols = ['paper_1_TOT','paper_2_TOT','paper_3_TOT','paper_4_TOT','paper_5_TOT','paper_6_TOT','paper_7_TOT','paper_8_TOT','paper_9_TOT','paper_10_TOT']

marks[TOT_cols] = marks[cols].apply(lambda x: x.map(lambda y: y[-6:-4]))

In [ ]:
EX_cols = ['paper_1_EX','paper_2_EX','paper_3_EX','paper_4_EX','paper_5_EX','paper_6_EX','paper_7_EX','paper_8_EX','paper_9_EX','paper_10_EX']

marks[EX_cols] = marks[cols].apply(lambda x: x.map(lambda y: y[0:4]))

Cleaning extra spaces and quotes

In [ ]:
for col in IA_cols:
    marks[col] = marks[col].map(lambda x: x.strip().strip("'"))
for col in EX_cols:
    marks[col] = marks[col].map(lambda x: x.strip().strip("'"))
for col in TOT_cols:
    marks[col] = marks[col].map(lambda x: x.strip().strip("'"))

In [ ]:
for col in IA_cols:
    marks[col] = marks[col].map(lambda x: x.strip())
for col in EX_cols:
    marks[col] = marks[col].map(lambda x: x.strip())
for col in TOT_cols:
    marks[col] = marks[col].map(lambda x: x.strip())

In [ ]:
marks.drop(cols,inplace = True,axis = 1)

In [ ]:
all_cols = IA_cols + EX_cols + TOT_cols

for col in all_cols:
    marks[col] = marks[col].map(lambda x: re.sub(r'\D','',x))

Convert the columns to appropriate data type

In [ ]:
for col in IA_cols+TOT_cols:
    marks[col] = pd.to_numeric(marks[col])

In [ ]:
for col in ['paper_1_EX','paper_2_EX','paper_6_EX','paper_7_EX','paper_8_EX']:
    marks[col] = pd.to_numeric(marks[col])

In [ ]:
for i,col in enumerate(['paper_3_EX','paper_4_EX','paper_5_EX','paper_9_EX','paper_10_EX']):
    marks[col] = marks.iloc[:,i+11] - marks.iloc[:,i+1]

In [ ]:
marks.head()

In [ ]:
df.drop(['sub1_10','marks1_10','grades1_10'],axis=1,inplace=True)

In [ ]:
df['clg_id'] = pd.to_numeric(df['clg_id'])

### After careful and thorough evaluation and study of the grading model that the university uses in it's system I've come to the following conclusion:
###     a)It sucks since just a difference of one mark could mean that you get 0.13 sgpi less,if you are at the boundry of the grade's bracket, while on the other hand someone could have a 9 mark difference between you and themselves yet still have the same sgpi. Which seems unfair.
###     b)The grades i extracted could be calculated from the marks and the credit for each subject, thus making another table for it is simply redundant.

In [ ]:
#grades = df[['seat_no','grades1_10']].copy()

Drop names to make data anonymous to keep students privacy.

In [ ]:
df.drop(['name','mother_name'],axis=1,inplace=True)

rename column to make it more understandable as they are actually gradepoints and not marks

In [ ]:
df.rename(columns={'agg_marks':'total_gradepoints'},inplace = True)

In [ ]:
df.head()

#### Export the data as csv, ready to be analysed

In [ ]:
df.to_csv('./data/result_clean.csv',index=False)
marks.to_csv('./data/marks.csv',index=False)
subject.to_csv('./data/subject.csv',index=False)